## **Import**

In [ ]:
from sqlalchemy import create_engine, MetaData, text, inspect
from sqlalchemy.schema import CreateTable
import pandas as pd
from dotenv import load_dotenv
import os

## **Input Parameters**

In [ ]:
# Cargar variables desde .env
load_dotenv()

# Obtener los valores
SOURCE_USER = os.getenv("SOURCE_DB_USER")
SOURCE_PASSWORD = os.getenv("SOURCE_DB_PASSWORD")
SOURCE_HOST = os.getenv("SOURCE_DB_HOST")
SOURCE_PORT = os.getenv("SOURCE_DB_PORT")
SOURCE_DATABASE = os.getenv("SOURCE_DB_NAME")

TARGET_USER = os.getenv("TARGET_DB_USER")
TARGET_PASSWORD = os.getenv("TARGET_DB_PASSWORD")
TARGET_HOST = os.getenv("TARGET_DB_HOST")
TARGET_PORT = os.getenv("TARGET_DB_PORT")
TARGET_DATABASE = os.getenv("TARGET_DB_NAME")

## **Functions**

In [ ]:
def dql_execution(query_text, engine):
    df = pd.read_sql(query_text, engine)
    print(df)

In [ ]:
def ddl_dml_execution(sql_string,engine):
    with engine.connect() as connection:
        connection.execute(text(s|ql_string))
        connection.commit()  

## **Process**

In [ ]:
source_engine = create_engine(f"postgresql://{SOURCE_USER}:{SOURCE_PASSWORD}@{SOURCE_HOST}:5432/{SOURCE_DATABASE}")
target_engine = create_engine(f"postgresql://{TARGET_USER}:{TARGET_PASSWORD}@{TARGET_HOST}:5433/{TARGET_DATABASE}")

In [ ]:
source_inspector = inspect(source_engine)

In [ ]:
source_schemas = source_inspector.get_schema_names()

In [ ]:
source_schemas.remove("information_schema")

In [ ]:
for schema_name in source_schemas:
    metadata = MetaData(schema=schema_name)
    metadata.reflect(bind=source_engine)
    ddl_dml_execution(f"CREATE SCHEMA IF NOT EXISTS {schema_name};",target_engine)    
    tables = source_inspector.get_table_names(schema=schema_name)

    for num,table in enumerate(metadata.sorted_tables):
        ddl_dml_execution(f"{str(CreateTable(table).compile(source_engine))};",target_engine)